In [ ]:
# code to manually check performance of a model on validation set
# !!! skeleton code
import model
import pan_loader
import base_config
import loss_functions as L


import visualize
import utils

import torch
import numpy as np

import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
data_dir = "/home/aravind/dataset/"
ann_dir = data_dir + "annotations/panoptic/"

train_img_dir = data_dir + "train2017/"
train_seg_dir = ann_dir + "panoptic_train2017/"
train_ann_json = ann_dir + "panoptic_train2017.json"

val_img_dir = data_dir + "val2017/"
val_seg_dir = ann_dir + "panoptic_val2017/"
val_ann_json = ann_dir + "panoptic_val2017.json"

train_img_dir = val_img_dir 
train_seg_dir = val_seg_dir 
train_ann_json = val_ann_json 

In [ ]:
with open(val_ann_json,"r") as f:
    val_ann = json.load(f)
with open(train_ann_json,"r") as f:
    train_ann = json.load(f)

In [ ]:
class InferenceConfig(base_config.Config):
    IMAGES_PER_GPU = 1
config = InferenceConfig()

In [ ]:
train_loader = pan_loader.get_loader(train_img_dir, train_seg_dir, train_ann, config)
val_loader = pan_loader.get_loader(val_img_dir, val_seg_dir, val_ann, config)

In [ ]:
model_dir = "models/"
model_name = "first_0.pt"

In [ ]:
net = model.hgmodel()

pretrained_dict = torch.load(model_dir+model_name)
net_dict = net.state_dict()

pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

net.eval()
net = net.cuda()

In [ ]:
from PIL import Image
for i, data in enumerate(val_loader,0):
    print("Image:\t%d"%i)
    images, impulses, instance_masks, cat_ids = utils.cudify_data(data)
    
    with torch.no_grad():
        outs = net([images,impulses])
    
    pred_masks, pred_class = outs
#     print(pred_masks.shape, instance_masks[0].shape)
    pred_masks = (pred_masks.squeeze().sigmoid()>0.5).cpu().numpy()
    Image.fromarray((pred_masks[0]*255).astype(np.uint8),"L").show()
    pred_labels = pred_class.argmax(1).cpu().numpy()
    # only first item in the batch
    images, impulses, instance_masks, cat_ids = [d[0] for d in data]
    visualize.visualize_data(images, impulses, instance_masks, cat_ids, config, "in")
    visualize.visualize_data(images, impulses, pred_masks, pred_labels, config, "out")
    input()

In [ ]:
pred_masks.shape